In [ ]:
import pandas as pd

In [ ]:
file = "data/181206_Datenbank_16pD.xlsx"

In [ ]:
df = pd.read_excel(file).fillna('FALSE')

In [ ]:
bildtechnik, _ = SkosCollection.objects.get_or_create(name="Bildtechnik", label_lang="ger")
bildmotiv, _ = SkosCollection.objects.get_or_create(name="Bildmotiv", label_lang="ger")

In [ ]:
for i, row in df.iterrows():
    try:
        lfnr = int(row['Lfd. Nr.'])
    except ValueError:
        lfnr = None
    if lfnr:
        card, _ = Card.objects.get_or_create(legacy_id=lfnr)

        if row['Serien-kürzel'] != 'FALSE':
            coll, _ = CardCollection.objects.get_or_create(abbreviation=row['Serien-kürzel'])
            card.card_collection = coll
        if row['Nr (Serie)'] != 'FALSE':
            card.number = row['Nr (Serie)']
        
        if row["Text Bildseite (Zitat)"] != 'FALSE':
            card.text_front = row["Text Bildseite (Zitat)"]
        if row["Text Adressseite (Zitat)"] != 'FALSE':
            card.text_back = row["Text Adressseite (Zitat)"]
        
        if row["Künstler (Bild)"] != 'FALSE':
            per, _ = Person.objects.get_or_create(written_name=row["Künstler (Bild)"])
            card.creator_person.add(per)

        if row['Bildtechnik'] != 'FALSE':
            bt, _ = SkosConcept.objects.get_or_create(
                pref_label=row['Bildtechnik'],
                pref_label_lang='ger'
            )
            bt.collection.add(bildtechnik)
            card.bild_technik.add(bt)
        
        if row['Index, Bildmotive'] != 'FALSE':
            for y in row['Index, Bildmotive'].split(', '):
                conc, _ = SkosConcept.objects.get_or_create(
                    pref_label=y.strip(),
                    pref_label_lang='ger'
                )
                conc.collection.add(bildmotiv)
                card.subject_norm.add(conc)
        
        if row['gelaufen / Datum'] != 'FALSE':
            card.gelaufen = row['gelaufen / Datum']
        
        if row['Ausgabe-datum'] != 'FALSE':
            card.written_date = row['Ausgabe-datum']
        else:
            card.written_date=""
        
        if row['Formale Anmerkungen'] != 'FALSE':
            card.note = row['Formale Anmerkungen']
        
        if row['Sonstige Anmerkungen'] != 'FALSE':
            card.note_content = row['Sonstige Anmerkungen']
         
        if row['Bildquelle'] != 'FALSE':
            arch = row['Bildquelle'].split(',')
            arch_inst, _ = Institution.objects.get_or_create(
                written_name=arch[0].strip(),
                institution_type='Archiv'
            )
            card.archiv = arch_inst
            card.signature = row['Bildquelle'].strip()
        
        if row['Index, Orte (nur anhand der Aufschriften)'] != 'FALSE':
            pl, _ = Place.objects.get_or_create(name=row['Index, Orte (nur anhand der Aufschriften)'])
            card.mentioned_place.add(pl)

        card.save()
        

In [ ]:
for x in Person.objects.all():
    try:
        x.forename = x.written_name.split(', ')[1]
        x.name = x.written_name.split(', ')[0]
        x.save()
    except:
        print(x.written_name)

In [ ]:
for x in Place.objects.all():
    x.name = x.name.replace('(Ort)', '')
    x.save()